In [40]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide_dataSet2.csv"
#device = "102"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"data/dataSet2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo', 'lstm_score'])

df = df.drop(columns=['Unnamed: 15'], errors='ignore')
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] >= 0]
df = df[df['rain'] >= 0]
df = df[(df['geo'] >= 0) & (df['geo'] < 50)]
df = df[df['temp'] >= 1]
df = df[(df['devID'] > 0) & (df['devID'] < 1000)]
df = df[df['timestamp'] > '2025-09-11']

#df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_24744\400936815.py:13: DtypeWarning: Columns (2,5,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk,confidence,rf_risk
166198,2025-09-11 00:00:01.157,103,17.700001,0.0,27.750000,83.849998,9.382449,8.640352,99.911812,0.068957,0,0.116043,0,NaN,NaN
166199,2025-09-11 00:00:16.183,103,17.700001,0.0,27.799999,83.099998,0.322535,8.640352,99.911812,0.076844,0,0.116043,0,NaN,NaN
166200,2025-09-11 00:00:31.220,103,17.700001,0.0,27.799999,83.099998,0.322535,8.640352,99.911812,0.076844,0,0.116043,0,NaN,NaN
166201,2025-09-11 00:00:46.316,103,17.700001,0.0,28.100000,83.000000,0.472299,8.640352,99.911812,0.074236,0,0.116043,0,NaN,NaN
166202,2025-09-11 00:01:01.346,103,17.700001,0.0,27.950000,83.049999,0.397417,8.640352,99.911812,0.074221,0,0.104980,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260995,2025-10-21 02:47:21.535,107,0.000000,0.0,29.500000,76.000000,0.561675,8.638103,99.898270,0.079643,0,0.132934,0,0.999999,0.0
260996,2025-10-21 02:47:32.105,106,0.000000,0.0,29.799999,73.900002,18.538265,8.638472,99.898506,0.055758,0,0.018138,0,0.999999,0.0
260998,2025-10-21 02:47:52.495,104,19.700001,0.0,32.599998,62.099998,5.568803,8.640429,99.911758,0.052534,0,0.174430,0,0.999999,0.0
260999,2025-10-21 02:48:02.254,107,0.000000,0.0,29.500000,76.000000,0.448819,8.638103,99.898270,0.079643,0,0.132934,0,0.999999,0.0


In [41]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,lstm_score,confidence,rf_risk
count,89834,89834.000000,89834.000000,89834.000000,89834.000000,89833.000000,89834.000000,89834.000000,89834.000000,89834.000000,89834.000000,60717.000000,60717.000000
mean,2025-09-29 18:02:25.841437696,103.636674,18.811470,0.072996,28.244391,79.124989,7.542329,8.639726,100.598478,-156.825114,0.289310,0.962577,0.004266
min,2025-09-11 00:00:01.157000,103.000000,0.000000,0.000000,6.550000,24.228929,0.000000,1.135011,32.000000,-86325.800490,0.000509,0.000000,0.000000
25%,2025-09-17 23:23:52.980499968,103.000000,17.700001,0.000000,25.299999,67.900002,0.460859,8.640355,99.911743,-0.008732,0.018143,0.950023,0.000000
50%,2025-09-28 15:32:09.334500096,104.000000,19.000000,0.000000,27.100000,82.199997,2.999028,8.640409,99.911758,0.023238,0.043992,0.999583,0.000000
75%,2025-10-09 18:07:00.442999808,104.000000,20.400000,0.000000,31.500000,91.000000,13.309074,8.640432,99.911812,0.062411,0.113979,0.999999,0.000000
max,2025-10-21 02:48:12.072000,107.000000,27.100000,36.321999,37.050001,98.800003,49.464066,8.640481,9954.666992,4069.171573,187.449081,1.000000,1.000000
std,NaN,0.661004,4.532379,0.838307,3.439988,13.645378,8.592803,0.066251,67.431763,3773.395168,4.039674,0.065127,0.065173


In [42]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_risk', 'iso_score', 'lstm_risk']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk,confidence,rf_risk
166198,2025-09-11 00:00:01.157,103,17.700001,0.0,27.750000,83.849998,9.382449,8.640352,99.911812,0.068957,0,0.116043,0,NaN,NaN
166199,2025-09-11 00:00:16.183,103,17.700001,0.0,27.799999,83.099998,0.322535,8.640352,99.911812,0.076844,0,0.116043,0,NaN,NaN
166200,2025-09-11 00:00:31.220,103,17.700001,0.0,27.799999,83.099998,0.322535,8.640352,99.911812,0.076844,0,0.116043,0,NaN,NaN
166201,2025-09-11 00:00:46.316,103,17.700001,0.0,28.100000,83.000000,0.472299,8.640352,99.911812,0.074236,0,0.116043,0,NaN,NaN
166202,2025-09-11 00:01:01.346,103,17.700001,0.0,27.950000,83.049999,0.397417,8.640352,99.911812,0.074221,0,0.104980,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260995,2025-10-21 02:47:21.535,107,0.000000,0.0,29.500000,76.000000,0.561675,8.638103,99.898270,0.079643,0,0.132934,0,0.999999,0.0
260996,2025-10-21 02:47:32.105,106,0.000000,0.0,29.799999,73.900002,18.538265,8.638472,99.898506,0.055758,0,0.018138,0,0.999999,0.0
260998,2025-10-21 02:47:52.495,104,19.700001,0.0,32.599998,62.099998,5.568803,8.640429,99.911758,0.052534,0,0.174430,0,0.999999,0.0
260999,2025-10-21 02:48:02.254,107,0.000000,0.0,29.500000,76.000000,0.448819,8.638103,99.898270,0.079643,0,0.132934,0,0.999999,0.0


In [43]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
#df['hour'] = df.index.hourผ
#df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89834 entries, 0 to 89833
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   89834 non-null  datetime64[ns]
 1   devID       89834 non-null  int64         
 2   soil        89834 non-null  float64       
 3   rain        89834 non-null  float64       
 4   temp        89834 non-null  float64       
 5   humi        89833 non-null  float64       
 6   geo         89834 non-null  float64       
 7   lat         89834 non-null  float64       
 8   lng         89834 non-null  float64       
 9   iso_score   89834 non-null  float64       
 10  iso_risk    89834 non-null  int64         
 11  lstm_score  89834 non-null  float64       
 12  lstm_risk   89834 non-null  int64         
 13  confidence  60717 non-null  float64       
 14  rf_risk     60717 non-null  float64       
dtypes: datetime64[ns](1), float64(11), int64(3)
memory usage: 10.3 MB


In [45]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
humi              1
confidence    29117
rf_risk       29117
dtype: int64


In [46]:
df = df.dropna(subset=['humi']).reset_index(drop=True)

In [47]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
confidence    29117
rf_risk       29117
dtype: int64


In [48]:
#df = df.dropna()
#
#nan_counts = df.isna().sum()
#nan_columns = nan_counts[nan_counts > 0]
#print("df102")
#print("📌 คอลัมน์ที่มีค่า NaN:")
#print(nan_columns)

In [49]:
#df[(df["iso_risk"]==2) & (df["lstm_risk"]==2)]

In [50]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk,confidence,rf_risk
count,89833,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,89833.000000,60716.000000,60716.000000
mean,2025-09-29 18:02:05.908472832,103.636670,18.811456,0.072997,28.244405,79.124989,7.542407,8.639726,100.598486,-156.826860,0.062616,0.289313,0.007380,0.962577,0.004266
min,2025-09-11 00:00:01.157000,103.000000,0.000000,0.000000,6.550000,24.228929,0.000000,1.135011,32.000000,-86325.800490,0.000000,0.000509,0.000000,0.000000,0.000000
25%,2025-09-17 23:23:48.092000,103.000000,17.700001,0.000000,25.299999,67.900002,0.460859,8.640355,99.911743,-0.008733,0.000000,0.018144,0.000000,0.950023,0.000000
50%,2025-09-28 15:32:04.468999936,104.000000,19.000000,0.000000,27.100000,82.199997,2.999200,8.640409,99.911758,0.023238,0.000000,0.043992,0.000000,0.999583,0.000000
75%,2025-10-09 18:06:46.096999936,104.000000,20.400000,0.000000,31.500000,91.000000,13.309219,8.640432,99.911812,0.062411,0.000000,0.113981,0.000000,0.999999,0.000000
max,2025-10-21 02:48:12.072000,107.000000,27.100000,36.321999,37.050001,98.800003,49.464066,8.640481,9954.666992,4069.171573,2.000000,187.449081,2.000000,1.000000,1.000000
std,NaN,0.661007,4.532402,0.838312,3.440005,13.645378,8.592820,0.066251,67.432139,3773.416134,0.263152,4.039696,0.110058,0.065128,0.065174


In [51]:
df['devID'].value_counts()

devID
104    47570
103    38673
105     1903
106      930
107      757
Name: count, dtype: int64

In [52]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
